# Twitter Bot

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import tweepy
import time
import os

# Import and setup Vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Finding API Keys
api_dir = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
file_name = os.path.join(api_dir, "api_keys.json")
data = json.load(open(file_name))

# Setting up API Key access
consumer_key = data['twitter_consumer_key']
consumer_secret = data['twitter_consumer_secret']
access_token = data['twitter_access_token']
access_token_secret = data['twitter_access_token_secret']

# Setting up Tweepy API Auth
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

# Globals

In [2]:
# Defining bot handle
bot_acc = '@FlawPlotBot'

# Setting up easy variable for testing
wait = 5*60

# Setting up a variable to change tweet amount
tweet_analyze_count = 500

# Setting up repeat bool
repeat = False

# Setting up list for repeated tags
repeat_tags = []

# Creating Functions

In [3]:
# Grabbing tweets and creating a plot
def runPrint(target_tag):
    
    # Creates a DF from target tag's tweets
    df = runVader(target_tag)
    
    # Creates a plot based on each item passed in from the DF
    # Formatting --- Pre-data
    plt.style.use('seaborn')
    colorlist = ["#e6194b", "#3cb44b", "#0082c8", "#f58231", "#911eb4", "#46f0f0", "#f032e6", "#ffe119", "#d2f53c", "#fabebe", "#000080", "#ffd8b1", "#808000", "#aaffc3", "#800000", "#fffac8", "#aa6e28", "#e6beff", "#008080"]
    plt.figure(figsize = (10, 5))
    
    # Setting up a plot
    plt.scatter(df.index, df['Compound'], color = 'b', marker = 'o', s=100, edgecolors="black", alpha=.75)
    
    # Formatting --- Post-data
    plt.xlim(-5, 505)
    plt.ylim(-1.01,1.01)
    plt.legend(loc = 'best')
    plt.xlabel('Tweets Ago', fontsize=16)
    plt.ylabel('Tweet Polarity', fontsize=16)
    plt.title(f'Sentiment Analysis for @{target_tag}', fontsize=22)
    #plt.grid(alpha = .25)
    
    # Saves figure for later report
    plt.savefig(f'Resources/Sentiment Tweet Analysis - @{target_tag} ({tweet_analyze_count} Tweets).png')

In [4]:
# Runs a check on if the requested tag has been seen twice
def runRepeat(target_tag):
    # Setting repeat bool to False to start
    repeat = False
    if target_tag in repeat_tags:
        return(True)
    else:
        repeat_tags.append(target_tag)
        return(False)

In [5]:
# Runs Vader Analysis on Tweets based on a tag passed in
def runVader(target_tag):
    
    # Setting variables
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    date_list = []
    tweet_text_list = []
    
    # Loop through pages based on total count (20 tweets per page)
    for x in range(int(tweet_analyze_count/20)):

        # Get all tweets from home feed  --  Page=x was giving double 1st page results
        public_tweets = api.user_timeline(target_tag, page=x+1)

        # Loop through all tweets 
        for tweet in public_tweets:
            
            # Run Vader Analysis on each tweet and add values to the appropriate arrays
            compound_list.append(analyzer.polarity_scores(tweet["text"])["compound"])
            positive_list.append(analyzer.polarity_scores(tweet["text"])["pos"])
            negative_list.append(analyzer.polarity_scores(tweet["text"])["neg"])
            neutral_list.append(analyzer.polarity_scores(tweet["text"])["neu"])
            date_list.append(tweet["created_at"])
            tweet_text_list.append(tweet["text"])
            
    # Pushing results to a dict       
    sentiment = {
                "Date": date_list,
                "Compound": compound_list,
                "Positive": positive_list,
                "Neutral": negative_list,
                "Negative": neutral_list,
                "Text": tweet_text_list
    }
    
    # Creating a DF for results
    result = pd.DataFrame(sentiment)
    
    # Optional - Prints progress of API Calls
    #print('--------------------------------------------------------')
    #print(f'Processing Twitter handle: {target_tag}')
    #print('--------------------------------------------------------')
    
    # Returning a DF of the results
    return(result)

In [6]:
# Setting up function to call so sleep can be put inside a wait loop on a timer
def runTweeter(bot_acc):
    
    # Grabbing only most recent tweet - No queueing system
    mentions = api.home_timeline(count=1)
    
    # Clearing target tag
    target_tag = ""

    # Creating a try "loop" to check if the most recent status is valid
    try:
        target_tag = mentions[0]['entities']['user_mentions'][1]['screen_name']
        
    # Reseting target_tag if it fails
    except:
        target_tag = ""

    # Starting IF block to grab rest of info if needed
    if (mentions[0]['text'] == f'{bot_acc} Analyze: @{target_tag}'):

        # Grab who requested for tweet-back
        req_user = mentions[0]['user']['screen_name']

        # Optional - Confirm target_account
        #print("Target Account: " + target_tag)
        #print("Requesting User: " + req_user)

    # Return both users
    return(target_tag, req_user)     

In [7]:
# Function to update after succesful analyze
def runSuccessUpdate(target_tag, req_user):
    api.update_with_media(f'Resources/Sentiment Tweet Analysis - @{target_tag} ({tweet_analyze_count} Tweets).png', f'@{req_user} Result for your request of handle: @{target_tag}')
    

# Main Execution

In [ ]:
# Loops and waits - Check for repeat tags and only updates on new tags
while(True):
    
    # Finding Users needed
    target_tag, req_user = runTweeter(bot_acc)
    
    # Repeat Check
    repeat = runRepeat(target_tag)
    
    # Execute if not a repeated user
    if not (repeat):
        runPrint(target_tag)
        runSuccessUpdate(target_tag, req_user)
    
    # Optional - Prints Repeated tags stored each cycle
    #print(repeat_tags)
            
    # Sleeping Function so as to not spam
    time.sleep(wait)

['CNN']
['CNN', 'espn']
['CNN', 'espn', 'NFL']
['CNN', 'espn', 'NFL', 'KingJames']
